# Interactive tool for analyzing trained models on the ePodium dataset

1. [Input Data](#1ma)
2. [Deep Learning Model](#2ma)
3. [Make predictions on testing set](#3ma)

#### Import Packages
Note: This notebook may output tensorflow errors if cuda is not properly installed. The notebook still functions with these errors.

In [ ]:
import mne
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import ipywidgets
import tensorflow as tf
import pandas as pd
from IPython.display import clear_output

# Local
import local_paths
from functions import epodium, display_helper
from functions.epodium import Epodium
from functions.ddp import DDP
from functions.train_and_predict import EpodiumSequence, DDPSequence

# Models
from models.dl_4_tsc import encoder_model, fully_convolutional_model, resnet_model
from models.eeg_dl import transformer_model

<br>

<a id="1ma"></a>
## 1. Input Data 

####  Choose which processed data to use
Choose from the different processed _experiment_event.npy_ files. If the _local_path.split_ folder is empty, process the raw ePodium files in the _epodium_processing_ notebook.

In [ ]:
# Choose between datasets: "epodium" "ddp"
dataset_name = "ddp"

if dataset_name == "epodium":
    dataset = Epodium()    
    epochs_directory = local_paths.ePod_epochs
    event_directory = local_paths.ePod_epochs_events
    
    epod_labels = dataset.create_labels(local_paths.ePod_metadata)
    print(f"The available labels are:\n {list(epod_labels.columns)}")

elif dataset_name == "ddp":
    dataset = DDP()
    epochs_directory = local_paths.DDP_epochs
    event_directory = local_paths.DDP_epochs_events
    
    directory_age_metadata = os.path.join(local_paths.DDP_metadata, "ages")
    ddp_labels = dataset.create_labels(local_paths.DDP_dataset, directory_age_metadata)
    print(f"The available labels are:\n {list(ddp_labels.columns)}") 

<br>

<a id="2ma"></a>
## 2. Deep Learning Model

#### Choose a trained model
Choose from the trained models in the _local_paths.models_ folder. If the folder is empty, train a model in the _epodium_model_training_ notebook.

In [ ]:
# Required plot setting
%matplotlib inline 
trained_models = sorted(f for f in os.listdir(os.path.join(local_paths.models)) if not f.startswith("."))

model_widget = ipywidgets.RadioButtons(options=trained_models, description='Models:')
display(model_widget)

history = []
                                       
def load_model(mod):
    base_path = os.path.join(local_paths.models, mod)

    path_history = os.path.join(base_path, "history.npy")
    path_model = os.path.join(base_path, "model")
    path_testset = os.path.join(base_path, "subsets", "test_set.txt")
    path_weights = os.path.join(base_path, "weights.h5")
    
    global model
    global testset
    global history

    # Load Model
    if(os.path.exists(path_model)):
        print(f"Loading Model: '{model_widget.value}'.")

        # Loads the entire model from a folder:
        model = tf.keras.models.load_model(path_model)
        model.load_weights(path_weights)
        # Reads the test-set of the trained model and puts the experiment names into a list:
        testset = open(path_testset, "r").read().split()
        # Loads the training history dictionary:
        history = np.load(path_history, allow_pickle=True).item()
        
        # Show Loss of Training History
        display_helper.show_plot(x=range(len(history['loss'][:])), y=history['loss'][:],  ylim=[0,600000], 
                                 xlabel="epochs", ylabel="validation loss", title="Loss during training")
        display_helper.show_plot(x=range(len(history['loss']))[:], y=history['val_loss'][:], ylim=[0,600000],
                                 xlabel="epochs", ylabel="validation loss", title="Validation loss during training")
        print(f"The lowest validation loss is {round(min(history['val_loss']), 3)}")

    else: 
        print("Model not found")

out = ipywidgets.interactive_output(load_model, {'mod': model_widget})
display(out)

<br>

<a id='3ma'></a>
## 3. Make predictions on testing set


In [ ]:
n_passthroughs = 2
results = []

# For each experiment in the test-set
for i, experiment in enumerate(testset):
    test_sequence = DDPSequence([experiment], 
                                ddp_labels, 
                                epochs_directory, 
                                n_experiments_batch=1, 
                                n_instances_per_experiment=n_passthroughs)
    x, y = test_sequence.__getitem__(0)
    
    # Make a prediction with the model.
    real_pred = [y[0], np.squeeze(model.predict(x, verbose=0)).mean()]
    results.append(real_pred)
    
    print(f"{i+1}/{len(testset)} predicted.")
    clear_output(wait=True)
    
results = np.array(results)

In [ ]:
display_helper.show_plot(np.array(results)[:,0], 
                         np.array(results)[:,1], 
                         f"Age prediction (encoder_age_128_20, passes: {n_passthroughs})", 
                         "Actual age (days)",
                         "Predicted age (days)",
                         scatter=True,
                         show=False)
plt.grid()
plt.plot([300, 1450], [300, 1450]) # Line where predicted=actual